### AGE & GENDER DETECTION MULTITASK MODEL
This model helps us to make our single custom neural network that can simultaneously do two seperate tasks of detecting gender and age at the same time.

In [1]:
import pandas as pd 
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer
import matplotlib.pyplot as plt
from random import shuffle
import os
from keras.models import Model, Sequential, load_model
from keras import layers
from keras.layers import Convolution2D, MaxPooling2D, Dropout, Flatten, Dense, Activation,GlobalMaxPooling2D, ZeroPadding2D
from keras import applications
#from keras.utils import plot_model, to_categorical
from keras.applications import VGG16, ResNet50
from keras.optimizers import RMSprop,Adam  
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
import pickle
import random
import cv2

Using TensorFlow backend.


In [2]:
path = "train/"
files = os.listdir(path)
size = len(files)
shuffle(files)
print("Total samples:",size)
print(files[0])

Total samples: 23708
28_0_3_20170119194507123.jpg.chip.jpg


In [5]:
#files = os.listdir("train")
#shuffle(files)
ages=[]  #Children (1-14) CLASS 0;Youth (14-25) CLASS 1;ADULTS (25-40) CLASS 2;Middle age (40-60) CLASS 3
         #Very Old (>60) CLASS 4
genders=[] #0=male ;1=female
images=[]
for file in files:
    image= cv2.imread(path+file,1) #1 means colourful
    image = cv2.resize(image,dsize=(32,32))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = img_to_array(image)
    #image = image.reshape((image.shape[0],image.shape[1],3))
    images.append(image)
    split_var = file.split('_')
    genders.append(int(split_var[1]) )
    label_1=file.split('_')[0]
    label_1 = int(label_1)
    if label_1 <= 14:
         ages.append(('0'))
    if (label_1>14) and (label_1<=25):
        ages.append(('1'))
    if (label_1>25) and (label_1<40):
        ages.append(('2'))
    if (label_1>=40) and (label_1<60):
        ages.append(('3'))
    if label_1>=60:
        ages.append(('4'))
   

In [6]:
#print(images[7])
print(ages[7])
print(genders[7])


4
1


In [7]:
images = np.array(images, dtype="float") / 255.0
 
ages = np.array(ages)
genders = np.array(genders)
 
agesLB = LabelBinarizer()
gendersLB = LabelBinarizer()

ages = agesLB.fit_transform(ages)
genders = gendersLB.fit_transform(genders)

In [8]:
split = train_test_split(images, ages, genders, test_size=0.2, random_state=42)
(trainX, testX, trainAgeY, testAgeY, trainGenderY, testGenderY) = split

In [9]:

print(agesLB.classes_)
print(gendersLB.classes_)

['0' '1' '2' '3' '4']
[0 1]


In [18]:
img_row = 32
img_col = 32
epochs = 50
batch_size = 128

In [11]:
pre_trained_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_row,img_col,3)) #taking pretrained weights;top layer chopped off as i am going to use it of my own;
    
for layer in pre_trained_model.layers[:15]:
    layer.trainable = False  #freeze upto 15th layer only of vgg16

for layer in pre_trained_model.layers[15:]:
    layer.trainable = False  #if true fine tuning vgg16 starts from this layer

In [ ]:
###OWN MODEL FROM SCRATCH

# import keras 
# from keras.layers import *
# from keras.models import *
# from keras import backend as K

# visible=Input(shape=(32,32,3))
# x=Convolution2D(64, (3, 3), padding = 'same', input_shape = (img_row, img_col, 3))(visible)
# x=Activation('relu')(x)
# x=BatchNormalization()(x)

# # Second CONV-ReLU Layer
# x=Convolution2D(64, (3, 3), padding = 'same', input_shape = (img_row, img_col, 3))(x)
# x=Activation('relu')(x)
# x=BatchNormalization()(x)

# # Max Pooling with Dropout 
# x=MaxPooling2D(pool_size=(2, 2))(x)
# x=Dropout(0.2)(x)

# # 3rd set of CONV-ReLU Layers
# x=Convolution2D(128, (3, 3), padding = 'same', input_shape = (img_row, img_col, 3))(x)
# x=Activation('relu')(x)
# x=BatchNormalization()(x)
# # 4th Set of CONV-ReLU Layers
# x=Convolution2D(128, (3, 3), padding = 'same', input_shape = (img_row, img_col, 3))(x)
# x=Activation('relu')(x)
# x=BatchNormalization()(x)
# # Max Pooling with Dropout 
# x=MaxPooling2D(pool_size=(2, 2))(x)
# x=Dropout(0.2)(x)
# # 5th Set of CONV-ReLU Layers
# x=Convolution2D(256, (3, 3), padding = 'same', input_shape = (img_row, img_col, 3))(x)
# x=Activation('relu')(x)
# x=BatchNormalization()(x)

# # 6th Set of CONV-ReLU Layers
# x=Convolution2D(256, (3, 3), padding = 'same', input_shape = (img_row, img_col, 3))(x)
# x=Activation('relu')(x)
# x=BatchNormalization()(x)

# # Max Pooling with Dropout 
# x=MaxPooling2D(pool_size=(2, 2))(x)
# top_model=Dropout(0.2)(x)

In [12]:
top_model = pre_trained_model.output #output of vgg16 last layer taken as input here
top_model = Flatten()(top_model)
layer_1 = Dense(1024,activation='relu')(top_model)
layer_2= Dense(1024,activation='relu')(top_model)
layer_1 = Dense(128,activation='relu')(layer_1)
layer_1 = Dropout(0.5)(layer_1)
layer_2 = Dropout(0.5)(layer_2)
out_age = Dense(5,activation='softmax',name='out_age')(layer_1) #for age
out_gender = Dense(1,activation='sigmoid',name='out_gender')(layer_2) #for gender

In [13]:
model = Model(inputs = pre_trained_model.input ,outputs=[out_age,out_gender])

In [34]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 32, 32, 64)   1792        input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 32, 32, 64)   36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 16, 16, 64)   0           block1_conv2[0][0]               
____________________________________________________________________________________________

In [14]:
model.compile(optimizer='Adam',                              #{'out_1':'adam','out_2':'adam'}, 
              loss={'out_age':'categorical_crossentropy','out_gender': 'binary_crossentropy'},
              metrics ={'out_age': 'mae', 'out_gender': 'accuracy'}                                #{'out_1':"accuracy",'out_2':"accuracy"}
             )

In [19]:
checkpoint = ModelCheckpoint("multitask_age_gender_final.h5",
                             monitor="val_accuracy",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 15,
                          verbose = 1,
                          restore_best_weights = True)

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 3, verbose = 1, min_delta = 0.0001)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint, reduce_lr]

In [20]:
H = model.fit(trainX,
	{"out_age": trainAgeY, "out_gender": trainGenderY},
	validation_data=(testX,
		{"out_age": testAgeY, "out_gender": testGenderY}),
	epochs=epochs,
    callbacks=callbacks,
	verbose=1)

Train on 18966 samples, validate on 4742 samples
Epoch 1/50
18966/18966 [==============================] - 44s 2ms/step - loss: 1.8455 - out_age_loss: 1.2928 - out_gender_loss: 0.5525 - out_age_mae: 0.2653 - out_gender_accuracy: 0.7220 - val_loss: 1.6968 - val_out_age_loss: 1.1765 - val_out_gender_loss: 0.5221 - val_out_age_mae: 0.2472 - val_out_gender_accuracy: 0.7421
Epoch 2/50
  160/18966 [..............................] - ETA: 19s - loss: 1.6998 - out_age_loss: 1.2268 - out_gender_loss: 0.4730 - out_age_mae: 0.2554 - out_gender_accuracy: 0.7437

D:\TOOLS\Anaconda\envs\tf\lib\site-packages\keras\callbacks\callbacks.py:707: RuntimeWarning: Can save best model only with val_accuracy available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


18966/18966 [==============================] - 22s 1ms/step - loss: 1.7006 - out_age_loss: 1.1878 - out_gender_loss: 0.5129 - out_age_mae: 0.2489 - out_gender_accuracy: 0.7511 - val_loss: 1.6997 - val_out_age_loss: 1.1826 - val_out_gender_loss: 0.5177 - val_out_age_mae: 0.2466 - val_out_gender_accuracy: 0.7442
Epoch 3/50
18966/18966 [==============================] - 22s 1ms/step - loss: 1.6490 - out_age_loss: 1.1549 - out_gender_loss: 0.4941 - out_age_mae: 0.2435 - out_gender_accuracy: 0.7585 - val_loss: 1.6227 - val_out_age_loss: 1.1425 - val_out_gender_loss: 0.4829 - val_out_age_mae: 0.2384 - val_out_gender_accuracy: 0.7655
Epoch 4/50
18966/18966 [==============================] - 22s 1ms/step - loss: 1.6147 - out_age_loss: 1.1288 - out_gender_loss: 0.4861 - out_age_mae: 0.2393 - out_gender_accuracy: 0.7664 - val_loss: 1.6077 - val_out_age_loss: 1.1330 - val_out_gender_loss: 0.4761 - val_out_age_mae: 0.2405 - val_out_gender_accuracy: 0.7659
Epoch 5/50
18966/18966 [==================

In [21]:
model.save("age_gender_multitask_1_23epoch")